##### Librerías a utilizar

In [18]:
import datetime
from collections import deque 
import copy 

## Modelado de Datos y Estructura del Proyecto

En esta primera sección, definimos las clases de Python que representarán las entidades fundamentales de nuestro problema:

*   **`Habilidad`**: Una competencia específica que un empleado puede tener (ej. "Enfermería").
*   **`Departamento`**: Un área de trabajo que requiere ciertas habilidades (ej. "Maternidad").
*   **`Empleado`**: Una persona con un conjunto de habilidades y una disponibilidad semanal.
*   **`Turno`**: Un bloque de trabajo que necesita ser asignado a un empleado. **Esta será la variable de nuestro CSP.**

Establecer estas clases de forma clara es crucial para construir una solución lógica y mantenible.

In [10]:
# Importaciones necesarias para type hinting (buenas prácticas)
from typing import List, Dict, Optional

# Definimos una constante para los días de la semana para evitar errores de tipeo
DIAS_SEMANA = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

class Habilidad:
    """Representa una habilidad única que un empleado puede poseer."""
    def __init__(self, nombre: str):
        self.nombre = nombre

    def __repr__(self) -> str:
        # Representación útil para debugging
        return f"Habilidad({self.nombre})"

    def __eq__(self, other):
        # Permite comparar habilidades por su nombre
        return isinstance(other, Habilidad) and self.nombre == other.nombre
    
    def __hash__(self):
        # Permite usar objetos Habilidad en sets o como llaves de diccionario
        return hash(self.nombre)


class Departamento:
    """Representa un departamento que tiene turnos y requiere habilidades específicas."""
    def __init__(self, nombre: str, habilidades_requeridas: List[Habilidad]):
        self.nombre = nombre
        self.habilidades_requeridas = habilidades_requeridas

    def __repr__(self) -> str:
        return f"Departamento({self.nombre})"


class Empleado:
    """Representa a un empleado con sus habilidades y su disponibilidad."""
    def __init__(self, nombre: str, habilidades: List[Habilidad], disponibilidad: Optional[Dict[str, bool]] = None):
        self.nombre = nombre
        self.habilidades = set(habilidades) # Usar un set para búsquedas más rápidas

        # Si no se especifica disponibilidad, se asume que está disponible todos los días
        if disponibilidad is None:
            self.disponibilidad = {dia: True for dia in DIAS_SEMANA}
        else:
            self.disponibilidad = disponibilidad

    def tiene_habilidad(self, habilidad_requerida: Habilidad) -> bool:
        """Verifica si el empleado posee una habilidad requerida."""
        return habilidad_requerida in self.habilidades

    def tiene_todas_las_habilidades(self, habilidades_requeridas: List[Habilidad]) -> bool:
        """Verifica si el empleado posee una lista completa de habilidades."""
        return all(self.tiene_habilidad(h) for h in habilidades_requeridas)

    def puede_trabajar(self, dia: str) -> bool:
        """Verifica si el empleado está disponible en un día específico."""
        return self.disponibilidad.get(dia, False)

    def __repr__(self) -> str:
        return f"Empleado({self.nombre})"


class Turno:
    """Representa un turno de trabajo. Esta es la VARIABLE en nuestro modelo CSP."""
    def __init__(self, id: str, dia: str, hora_inicio: int, hora_fin: int, departamento: Departamento):
        # El ID debe ser único para identificar cada turno
        self.id = id
        self.dia = dia
        self.hora_inicio = hora_inicio # Formato 24h, ej: 8 para 8:00
        self.hora_fin = hora_fin       # ej: 16 para 16:00
        self.departamento = departamento
        self.empleado_asignado: Optional[Empleado] = None

    def __repr__(self) -> str:
        # Representación detallada para fácil inspección
        return (f"Turno('{self.id}', {self.dia}, {self.hora_inicio:02d}:00-{self.hora_fin:02d}:00, "
                f"Dept: {self.departamento.nombre})")
    
    def __eq__(self, other):
        # Dos turnos son iguales si su ID es el mismo
        return isinstance(other, Turno) and self.id == other.id

    def __hash__(self):
        # Necesario para usar objetos Turno como llaves en diccionarios (para dominios, vecinos, etc.)
        return hash(self.id)

#### Prueba de las Clases: Creación de un Escenario de Ejemplo

Para asegurarnos de que nuestras clases funcionan como esperamos, vamos a crear un pequeño escenario de prueba con algunos empleados, departamentos y turnos. Imprimiremos los objetos y probaremos algunos de sus métodos.

In [11]:
# Definir Habilidades 
h_enfermeria = Habilidad("Enfermería")
h_pediatria = Habilidad("Pediatría")
h_cirugia = Habilidad("Cirugía")
h_basico = Habilidad("Cuidados Básicos")

# Definir Departamentos 
dep_maternidad = Departamento("Maternidad", [h_enfermeria, h_pediatria])
dep_urgencias = Departamento("Urgencias", [h_enfermeria, h_cirugia])
dep_general = Departamento("General", [h_basico])

# Definir Empleados 
# Ann: Enfermera pediátrica, disponible toda la semana
ann = Empleado("Ann", [h_enfermeria, h_pediatria])

# Beth: Enfermera quirúrgica, no trabaja fines de semana
disponibilidad_beth = {dia: True for dia in DIAS_SEMANA[:5]} # Lunes a Viernes
disponibilidad_beth.update({'Sábado': False, 'Domingo': False})
beth = Empleado("Beth", [h_enfermeria, h_cirugia], disponibilidad_beth)

# Cory: Solo cuidados básicos, disponible toda la semana
cory = Empleado("Cory", [h_basico])

# Dan: Enfermero general, sin especialidad
dan = Empleado("Dan", [h_enfermeria])


# 4. Definir Turnos
# Un ID único para cada turno es fundamental
turno1 = Turno("LUN-MANANA-MAT", "Lunes", 8, 16, dep_maternidad)
turno2 = Turno("LUN-TARDE-URG", "Lunes", 14, 22, dep_urgencias)
turno3 = Turno("MAR-NOCHE-GEN", "Martes", 22, 6, dep_general) # Turno nocturno que cruza al Miércoles
turno4 = Turno("SAB-DIA-MAT", "Sábado", 9, 17, dep_maternidad)


# 5. Realizar Pruebas
print("--- OBJETOS CREADOS ---")
print(ann)
print(dep_maternidad)
print(turno1)
print("-" * 25)

print("\n--- PRUEBAS DE MÉTODOS ---")
# ¿Ann tiene las habilidades para Maternidad?
print(f"¿Ann tiene habilidades para Maternidad?: {ann.tiene_todas_las_habilidades(dep_maternidad.habilidades_requeridas)}")
# ¿Beth tiene las habilidades para Maternidad? (Debería ser False, le falta Pediatría)
print(f"¿Beth tiene habilidades para Maternidad?: {beth.tiene_todas_las_habilidades(dep_maternidad.habilidades_requeridas)}")
# ¿Beth puede trabajar el Sábado?
print(f"¿Beth puede trabajar el Sábado?: {beth.puede_trabajar('Sábado')}")
# ¿Cory puede trabajar en Urgencias? (Debería ser False)
print(f"¿Cory tiene habilidades para Urgencias?: {cory.tiene_todas_las_habilidades(dep_urgencias.habilidades_requeridas)}")
print("-" * 25)

--- OBJETOS CREADOS ---
Empleado(Ann)
Departamento(Maternidad)
Turno('LUN-MANANA-MAT', Lunes, 08:00-16:00, Dept: Maternidad)
-------------------------

--- PRUEBAS DE MÉTODOS ---
¿Ann tiene habilidades para Maternidad?: True
¿Beth tiene habilidades para Maternidad?: False
¿Beth puede trabajar el Sábado?: False
¿Cory tiene habilidades para Urgencias?: False
-------------------------


## Creación del Solucionador CSP

Ahora creamos la clase `CSP_Solver`. Esta clase contendrá toda la lógica para resolver el problema de asignación.

*   **Variables**: Los `Turno` a asignar.
*   **Dominios**: Para cada `Turno`, la lista de `Empleado`s que podrían cubrirlo.
*   **Vecinos**: Un grafo que representa las restricciones entre pares de turnos.

In [12]:
class CSP_Solver:
    """
    Clase que encapsula la lógica para resolver el problema de asignación de turnos
    mediante técnicas de Satisfacción de Restricciones (CSP).
    """
    def __init__(self, turnos: List[Turno], empleados: List[Empleado]):
        self.variables: List[Turno] = turnos
        self.empleados: List[Empleado] = empleados
        
        # El dominio de una variable (turno) es la lista de valores (empleados) que puede tomar.
        # Lo almacenaremos en un diccionario: {turno_id: [empleado1, empleado2, ...]}
        self.dominios: Dict[str, List[Empleado]] = {}
        
        # Los vecinos de un turno son todos los otros turnos que tienen una restricción con él.
        # {turno_id: [vecino1, vecino2, ...]}
        self.vecinos: Dict[str, List[Turno]] = {}

    def inicializar_csp(self):
        """Método principal para preparar el CSP antes de resolverlo."""
        print("Inicializando CSP...")
        self._inicializar_dominios()
        self._construir_vecinos()
        print("CSP inicializado. Dominios y vecinos construidos.")
        
    def _inicializar_dominios(self):
        """
        Aplica las RESTRICCIONES UNARIAS.
        Para cada turno, su dominio inicial son los empleados que cumplen:
        1. Disponibilidad en el día del turno.
        2. Poseen todas las habilidades requeridas por el departamento del turno.
        """
        for turno in self.variables:
            dominio_valido = []
            for empleado in self.empleados:
                # Verificar disponibilidad del empleado
                if not empleado.puede_trabajar(turno.dia):
                    continue # Siguiente empleado
                
                # Verificar habilidades requeridas por el departamento
                habilidades_requeridas = turno.departamento.habilidades_requeridas
                if not empleado.tiene_todas_las_habilidades(habilidades_requeridas):
                    continue # Siguiente empleado
                
                # Si pasa ambas validaciones, el empleado es un candidato válido
                dominio_valido.append(empleado)
            
            self.dominios[turno.id] = dominio_valido

    def _construir_vecinos(self):
        """
        Aplica las RESTRICCIONES BINARIAS para determinar el grafo de restricciones.
        Dos turnos (t1, t2) son vecinos si un mismo empleado NO puede cubrirlos ambos.
        Esto ocurre si:
        1. Son en el mismo día (regla simple y estricta).
        2. Se solapan en el tiempo.
        3. No hay al menos 8 horas de descanso entre ellos.
        """
        for turno in self.variables:
            self.vecinos[turno.id] = []

        for i in range(len(self.variables)):
            for j in range(i + 1, len(self.variables)):
                t1 = self.variables[i]
                t2 = self.variables[j]

                # Condición 1: Mismo día (la más simple y restrictiva)
                # Esta regla ya cubre el solapamiento y el descanso en la mayoría de los casos no nocturnos.
                if t1.dia == t2.dia:
                    self.vecinos[t1.id].append(t2)
                    self.vecinos[t2.id].append(t1)

### Prueba de la Inicialización del CSP

Vamos a instanciar nuestro `CSP_Solver` con los datos de ejemplo que creamos anteriormente y llamaremos a `inicializar_csp()`. Luego, inspeccionaremos los `dominios` y `vecinos` generados para verificar que la lógica es correcta.

In [13]:
# Crear la lista de todos los empleados y turnos
todos_los_empleados = [ann, beth, cory, dan]

# Añadamos más turnos para hacer la prueba de vecinos más interesante
turno_lun_noche = Turno("LUN-NOCHE-URG", "Lunes", 22, 6, dep_urgencias) # Termina el Martes
turno_mar_manana = Turno("MAR-MANANA-URG", "Martes", 7, 15, dep_urgencias)

todos_los_turnos = [turno1, turno2, turno3, turno4, turno_lun_noche, turno_mar_manana]

# Crear e inicializar el solver 
solver = CSP_Solver(todos_los_turnos, todos_los_empleados)
solver.inicializar_csp()

# Imprimir y verificar los resultados
print("\n--- DOMINIOS INICIALES (RESTRICCIONES UNARIAS) ---")
for turno_id, dominio in solver.dominios.items():
    print(f"Turno '{turno_id}': {[empleado.nombre for empleado in dominio]}")

print("\n--- GRAFO DE VECINOS (RESTRICCIONES BINARIAS) ---")
for turno_id, lista_vecinos in solver.vecinos.items():
    print(f"Turno '{turno_id}' es vecino de: {[vecino.id for vecino in lista_vecinos]}")

Inicializando CSP...
CSP inicializado. Dominios y vecinos construidos.

--- DOMINIOS INICIALES (RESTRICCIONES UNARIAS) ---
Turno 'LUN-MANANA-MAT': ['Ann']
Turno 'LUN-TARDE-URG': ['Beth']
Turno 'MAR-NOCHE-GEN': ['Cory']
Turno 'SAB-DIA-MAT': ['Ann']
Turno 'LUN-NOCHE-URG': ['Beth']
Turno 'MAR-MANANA-URG': ['Beth']

--- GRAFO DE VECINOS (RESTRICCIONES BINARIAS) ---
Turno 'LUN-MANANA-MAT' es vecino de: ['LUN-TARDE-URG', 'LUN-NOCHE-URG']
Turno 'LUN-TARDE-URG' es vecino de: ['LUN-MANANA-MAT', 'LUN-NOCHE-URG']
Turno 'MAR-NOCHE-GEN' es vecino de: ['MAR-MANANA-URG']
Turno 'SAB-DIA-MAT' es vecino de: []
Turno 'LUN-NOCHE-URG' es vecino de: ['LUN-MANANA-MAT', 'LUN-TARDE-URG']
Turno 'MAR-MANANA-URG' es vecino de: ['MAR-NOCHE-GEN']


## Implementación del Algoritmo AC-3

Ahora implementaremos el algoritmo de consistencia de arcos (AC-3). Este algoritmo nos ayudará a reducir los dominios de cada variable (turno) eliminando valores (empleados) que no pueden ser parte de una solución consistente.

El algoritmo se compone de dos partes principales:
1.  **`revise(turno_i, turno_j)`**: La función que considera un solo arco y elimina valores del dominio de `turno_i` si no tienen soporte en `turno_j`.
2.  **`ac3()`**: El bucle principal que gestiona una cola de arcos a revisar hasta que el sistema se vuelve consistente.

In [16]:
class CSP_Solver:
    """
    Clase que encapsula la lógica para resolver el problema de asignación de turnos
    mediante técnicas de Satisfacción de Restricciones (CSP).
    AÑADIMOS AHORA LA LÓGICA DE AC-3.
    """
    def __init__(self, turnos: List[Turno], empleados: List[Empleado]):
        self.variables: List[Turno] = turnos
        self.empleados: List[Empleado] = empleados
        self.dominios: Dict[str, List[Empleado]] = {}
        self.vecinos: Dict[str, List[Turno]] = {}

    def inicializar_csp(self):
        print("Inicializando CSP...")
        self._inicializar_dominios()
        self._construir_vecinos()
        print("CSP inicializado. Dominios y vecinos construidos.")

    def _inicializar_dominios(self):
        for turno in self.variables:
            dominio_valido = []
            for empleado in self.empleados:
                if empleado.puede_trabajar(turno.dia) and \
                   empleado.tiene_todas_las_habilidades(turno.departamento.habilidades_requeridas):
                    dominio_valido.append(empleado)
            self.dominios[turno.id] = dominio_valido

    def _construir_vecinos(self):
        for turno in self.variables:
            self.vecinos[turno.id] = []
        dias_map = {dia: idx for idx, dia in enumerate(DIAS_SEMANA)}
        minutos_descanso = 8 * 60
        for i in range(len(self.variables)):
            for j in range(i + 1, len(self.variables)):
                t1, t2 = self.variables[i], self.variables[j]
                if t1.dia == t2.dia:
                    self.vecinos[t1.id].append(t2)
                    self.vecinos[t2.id].append(t1)
                    continue
                fin_t1_min = (dias_map[t1.dia]*24*60) + (t1.hora_fin*60)
                if t1.hora_fin < t1.hora_inicio: fin_t1_min += 24*60
                inicio_t2_min = (dias_map[t2.dia]*24*60) + (t2.hora_inicio*60)
                fin_t2_min = (dias_map[t2.dia]*24*60) + (t2.hora_fin*60)
                if t2.hora_fin < t2.hora_inicio: fin_t2_min += 24*60
                inicio_t1_min = (dias_map[t1.dia]*24*60) + (t1.hora_inicio*60)
                if abs(inicio_t2_min - fin_t1_min) < minutos_descanso or abs(inicio_t1_min - fin_t2_min) < minutos_descanso:
                    self.vecinos[t1.id].append(t2)
                    self.vecinos[t2.id].append(t1)

    # --- NUEVOS MÉTODOS PARA AC-3 ---

    def revise(self, turno_i: Turno, turno_j: Turno) -> bool:
        """
        Hace que el arco (turno_i, turno_j) sea consistente.
        Elimina valores del dominio de turno_i si no tienen un valor correspondiente en el dominio de turno_j.
        Devuelve True si se ha realizado una revisión (se ha eliminado algún valor).
        """
        revisado = False
        dominio_i = self.dominios[turno_i.id]

        for empleado_x in list(dominio_i): # Iteramos sobre una copia para poder modificar el original
            # Si para empleado_x no hay ningún valor posible en el dominio de turno_j, lo eliminamos.
            # "Posible" significa que la restricción se cumple. Aquí, la restricción es que el empleado sea diferente.
            # Si el dominio de turno_j solo contiene a empleado_x, entonces no hay soporte.
            if len(self.dominios[turno_j.id]) == 1 and self.dominios[turno_j.id][0] == empleado_x:
                dominio_i.remove(empleado_x)
                revisado = True
                
        return revisado

    def ac3(self) -> bool:
        """
        Algoritmo AC-3. Rellena una cola con todos los arcos del problema y los procesa
        hasta que no queden arcos por revisar.
        Devuelve False si se encuentra una inconsistencia (un dominio se vacía).
        Devuelve True si el proceso termina con éxito.
        """
        # 1. Inicializar la cola con todos los arcos del problema.
        # Un arco es una tupla (Turno_i, Turno_j) donde i y j son vecinos.
        cola = deque()
        for turno_id in self.vecinos:
            turno_i = next(t for t in self.variables if t.id == turno_id)
            for turno_j in self.vecinos[turno_id]:
                cola.append((turno_i, turno_j))

        # 2. Procesar la cola
        while cola:
            turno_i, turno_j = cola.popleft()

            if self.revise(turno_i, turno_j):
                # Si el dominio de turno_i se vacía, no hay solución.
                if not self.dominios[turno_i.id]:
                    return False
                
                # Si hemos podado el dominio de turno_i, debemos volver a revisar
                # todos los arcos que apuntan a turno_i.
                for turno_k in self.vecinos[turno_i.id]:
                    if turno_k != turno_j:
                        cola.append((turno_k, turno_i))
        
        return True # El CSP es consistente (o ha sido reducido).

In [17]:
print("--- PRUEBA DE AC-3 ---")

# --- 1. Escenario de prueba ---
# Dos turnos en el mismo día en el mismo departamento.
# Solo un empleado (Beth) tiene las habilidades, pero no puede hacer ambos turnos.
# AC-3 debería detectar que esto es un problema.
h_uci = Habilidad("UCI")
dep_uci = Departamento("UCI", [h_uci])

empleado_uci_1 = Empleado("Eva", [h_uci])
empleado_uci_2 = Empleado("Fabián", [h_uci])

turno_uci_1 = Turno("MIE-MANANA-UCI", "Miércoles", 8, 16, dep_uci)
turno_uci_2 = Turno("MIE-TARDE-UCI", "Miércoles", 16, 24, dep_uci)
# Añadimos un tercer turno para que sea más interesante
turno_uci_3 = Turno("JUE-MANANA-UCI", "Jueves", 8, 16, dep_uci)


# --- 2. Configurar y resolver con AC-3 ---
solver_ac3 = CSP_Solver(
    turnos=[turno_uci_1, turno_uci_2, turno_uci_3],
    empleados=[empleado_uci_1, empleado_uci_2]
)
solver_ac3.inicializar_csp()

print("\nDominios ANTES de AC-3:")
for turno_id, dominio in solver_ac3.dominios.items():
    print(f"Turno '{turno_id}': {[e.nombre for e in dominio]}")

# Ejecutar el algoritmo
resultado_ac3 = solver_ac3.ac3()

print("\n--- RESULTADOS DE AC-3 ---")
print(f"AC-3 terminó con éxito: {resultado_ac3}")

print("\nDominios DESPUÉS de AC-3:")
for turno_id, dominio in solver_ac3.dominios.items():
    print(f"Turno '{turno_id}': {[e.nombre for e in dominio]}")

# Verificación de la solución
solucionado_por_ac3 = True
if not resultado_ac3:
    solucionado_por_ac3 = False
else:
    for dominio in solver_ac3.dominios.values():
        if len(dominio) != 1:
            solucionado_por_ac3 = False
            break

print(f"\n¿Problema resuelto completamente por AC-3?: {solucionado_por_ac3}")

--- PRUEBA DE AC-3 ---
Inicializando CSP...
CSP inicializado. Dominios y vecinos construidos.

Dominios ANTES de AC-3:
Turno 'MIE-MANANA-UCI': ['Eva', 'Fabián']
Turno 'MIE-TARDE-UCI': ['Eva', 'Fabián']
Turno 'JUE-MANANA-UCI': ['Eva', 'Fabián']

--- RESULTADOS DE AC-3 ---
AC-3 terminó con éxito: True

Dominios DESPUÉS de AC-3:
Turno 'MIE-MANANA-UCI': ['Eva', 'Fabián']
Turno 'MIE-TARDE-UCI': ['Eva', 'Fabián']
Turno 'JUE-MANANA-UCI': ['Eva', 'Fabián']

¿Problema resuelto completamente por AC-3?: False


## Implementación de Backtrack Search (BTS) con Heurísticas

AC-3 es útil para podar el espacio de búsqueda, pero a menudo no es suficiente para encontrar una solución. Cuando los dominios de las variables aún contienen múltiples valores, necesitamos un algoritmo de búsqueda.

Implementaremos **Backtrack Search**, una forma de búsqueda en profundidad (DFS) que:
1.  Elige una variable no asignada usando la heurística **MRV (Minimum Remaining Values)**, que selecciona la variable con el dominio más pequeño. Esto nos permite fallar más rápido y podar ramas más grandes del árbol de búsqueda.
2.  Intenta asignar un valor de su dominio.
3.  Después de asignar, aplica **Forward Checking** para eliminar valores inconsistentes de los dominios de las variables vecinas.
4.  Si una asignación conduce a una solución, la devuelve. Si no, retrocede (backtracks) y prueba otro valor.

In [ ]:
class CSP_Solver:
    """
    Clase que encapsula la lógica para resolver el problema de asignación de turnos.
    AÑADIMOS AHORA LA LÓGICA DE BACKTRACKING (BTS).
    """
    def __init__(self, turnos: List[Turno], empleados: List[Empleado]):
        self.variables: List[Turno] = turnos
        self.empleados: List[Empleado] = empleados
        self.dominios: Dict[str, List[Empleado]] = {}
        self.vecinos: Dict[str, List[Turno]] = {}

    def inicializar_csp(self):
        print("Inicializando CSP...")
        self._inicializar_dominios()
        self._construir_vecinos()
        print("CSP inicializado. Dominios y vecinos construidos.")

    def _inicializar_dominios(self):
        for turno in self.variables:
            dominio_valido = []
            for empleado in self.empleados:
                if empleado.puede_trabajar(turno.dia) and \
                   empleado.tiene_todas_las_habilidades(turno.departamento.habilidades_requeridas):
                    dominio_valido.append(empleado)
            self.dominios[turno.id] = dominio_valido

    def _construir_vecinos(self):
        # (se queda igual)
        for turno in self.variables:
            self.vecinos[turno.id] = []
        dias_map = {dia: idx for idx, dia in enumerate(DIAS_SEMANA)}
        minutos_descanso = 8 * 60
        for i in range(len(self.variables)):
            for j in range(i + 1, len(self.variables)):
                t1, t2 = self.variables[i], self.variables[j]
                if t1.dia == t2.dia:
                    self.vecinos[t1.id].append(t2)
                    self.vecinos[t2.id].append(t1)
                    continue
                fin_t1_min = (dias_map[t1.dia]*24*60) + (t1.hora_fin*60)
                if t1.hora_fin < t1.hora_inicio: fin_t1_min += 24*60
                inicio_t2_min = (dias_map[t2.dia]*24*60) + (t2.hora_inicio*60)
                fin_t2_min = (dias_map[t2.dia]*24*60) + (t2.hora_fin*60)
                if t2.hora_fin < t2.hora_inicio: fin_t2_min += 24*60
                inicio_t1_min = (dias_map[t1.dia]*24*60) + (t1.hora_inicio*60)
                if abs(inicio_t2_min - fin_t1_min) < minutos_descanso or abs(inicio_t1_min - fin_t2_min) < minutos_descanso:
                    self.vecinos[t1.id].append(t2)
                    self.vecinos[t2.id].append(t1)

    def revise(self, turno_i: Turno, turno_j: Turno) -> bool:
        # ... (se queda igual)
        revisado = False
        dominio_i = self.dominios[turno_i.id]
        for empleado_x in list(dominio_i):
            if len(self.dominios[turno_j.id]) == 1 and self.dominios[turno_j.id][0] == empleado_x:
                dominio_i.remove(empleado_x)
                revisado = True
        return revisado

    def ac3(self) -> bool:
        # ... (se queda igual)
        cola = deque()
        for turno_id in self.vecinos:
            turno_i = next(t for t in self.variables if t.id == turno_id)
            for turno_j in self.vecinos[turno_id]:
                cola.append((turno_i, turno_j))
        while cola:
            turno_i, turno_j = cola.popleft()
            if self.revise(turno_i, turno_j):
                if not self.dominios[turno_i.id]: return False
                for turno_k in self.vecinos[turno_i.id]:
                    if turno_k != turno_j: cola.append((turno_k, turno_i))
        return True

    def backtrack(self, asignacion: Dict[str, Empleado]) -> Optional[Dict[str, Empleado]]:
        """Función principal de búsqueda recursiva."""
        # si la asignación está completa, hemos encontrado una solución.
        if len(asignacion) == len(self.variables):
            return asignacion

        # Selección de variable con MRV (Minimum Remaining Values)
        turno_a_asignar = self.seleccionar_variable_mrv(asignacion)
        
        # Iterar sobre los valores del dominio para la variable seleccionada
        for empleado_valor in self.dominios[turno_a_asignar.id]:
            # Guardamos una copia profunda de los dominios actuales por si necesitamos retroceder (backtrack)
            dominios_originales = copy.deepcopy(self.dominios)
            
            # Asignamos temporalmente
            asignacion[turno_a_asignar.id] = empleado_valor
            
            # Aplicamos Forward Checking
            # Si el forward checking no falla (no deja ningún dominio vacío)
            if self.forward_checking(turno_a_asignar, empleado_valor, asignacion):
                # 4. Llamada recursiva
                resultado = self.backtrack(asignacion)
                if resultado is not None:
                    return resultado
            
            # 5. Backtrack: Si la rama no llevó a una solución, deshacemos la asignación y restauramos los dominios.
            del asignacion[turno_a_asignar.id]
            self.dominios = dominios_originales

        # Si ningún valor funcionó, retornamos None (fallo)
        return None

    def seleccionar_variable_mrv(self, asignacion: Dict[str, Empleado]) -> Turno:
        """Elige la variable no asignada con el dominio más pequeño (MRV)."""
        variables_no_asignadas = [v for v in self.variables if v.id not in asignacion]
        
        # Encuentra la variable con el menor número de valores restantes en su dominio
        return min(variables_no_asignadas, key=lambda v: len(self.dominios[v.id]))

    def forward_checking(self, turno_actual: Turno, valor_asignado: Empleado, asignacion: Dict[str, Empleado]) -> bool:
        """
        Después de asignar un valor a un turno, elimina ese valor de los dominios de los vecinos no asignados.
        Devuelve False si algún dominio de un vecino se vacía, indicando un fallo.
        """
        for vecino in self.vecinos[turno_actual.id]:
            if vecino.id not in asignacion:
                if valor_asignado in self.dominios[vecino.id]:
                    self.dominios[vecino.id].remove(valor_asignado)
                    # Si al eliminar el valor, el dominio del vecino se queda vacío, esta rama no es viable.
                    if not self.dominios[vecino.id]:
                        return False
        return True

In [20]:
print("--- PRUEBA COMPLETA: AC-3 + BACKTRACKING ---")

# 1. Usamos el mismo escenario de la prueba anterior
h_uci = Habilidad("UCI")
dep_uci = Departamento("UCI", [h_uci])
empleado_uci_1 = Empleado("Eva", [h_uci])
empleado_uci_2 = Empleado("Fabián", [h_uci])
turno_uci_1 = Turno("MIE-MANANA-UCI", "Miércoles", 8, 16, dep_uci)
turno_uci_2 = Turno("MIE-TARDE-UCI", "Miércoles", 16, 24, dep_uci)
turno_uci_3 = Turno("JUE-MANANA-UCI", "Jueves", 8, 16, dep_uci)

# 2. Creamos el solver
solver_final = CSP_Solver(
    turnos=[turno_uci_1, turno_uci_2, turno_uci_3],
    empleados=[empleado_uci_1, empleado_uci_2]
)
solver_final.inicializar_csp()

print("\n--- PASO 1: Ejecutando AC-3 para podar dominios ---")
if not solver_final.ac3():
    print("El problema no tiene solución según AC-3.")
else:
    print("AC-3 completado. Dominios podados:")
    for turno_id, dominio in solver_final.dominios.items():
        print(f"  Turno '{turno_id}': {[e.nombre for e in dominio]}")

    # Verificar si AC-3 ya resolvió el problema
    solucionado_por_ac3 = all(len(d) == 1 for d in solver_final.dominios.values())
    
    if solucionado_por_ac3:
        print("\n¡Solución encontrada únicamente con AC-3!")
        solucion = {tid: d[0] for tid, d in solver_final.dominios.items()}
    else:
        print("\n--- PASO 2: AC-3 no fue suficiente. Iniciando Backtrack Search... ---")
        solucion = solver_final.backtrack({})

    # 3. Imprimir la solución final
    if solucion:
        print("\n¡SOLUCIÓN ENCONTRADA!")
        for turno_id, empleado in solucion.items():
            print(f"  Turno '{turno_id}' -> Asignado a: {empleado.nombre}")
    else:
        print("\nNo se pudo encontrar una solución que cumpla todas las restricciones.")

--- PRUEBA COMPLETA: AC-3 + BACKTRACKING ---
Inicializando CSP...
CSP inicializado. Dominios y vecinos construidos.

--- PASO 1: Ejecutando AC-3 para podar dominios ---
AC-3 completado. Dominios podados:
  Turno 'MIE-MANANA-UCI': ['Eva', 'Fabián']
  Turno 'MIE-TARDE-UCI': ['Eva', 'Fabián']
  Turno 'JUE-MANANA-UCI': ['Eva', 'Fabián']

--- PASO 2: AC-3 no fue suficiente. Iniciando Backtrack Search... ---

¡SOLUCIÓN ENCONTRADA!
  Turno 'MIE-MANANA-UCI' -> Asignado a: Eva
  Turno 'MIE-TARDE-UCI' -> Asignado a: Fabián
  Turno 'JUE-MANANA-UCI' -> Asignado a: Eva
